# The Stokes problem

On the unit square domain $\Omega$, we consider the Stokes equations

\begin{align*} 
- \boldsymbol{\nabla} \cdot (\boldsymbol{\nabla} \boldsymbol{u} - p I) &= \boldsymbol{f} \quad {\rm in} \ \Omega, \\ 
\boldsymbol{\nabla} \cdot \boldsymbol{u} &= 0 \quad {\rm in} \ \Omega. \\ 
\end{align*}

A typical set of boundary conditions on the boundary $\partial \Omega = \Gamma_{D} \cup \Gamma_{N}$ can be:
\begin{align*} \boldsymbol{u} &= \boldsymbol{u}_0 \quad {\rm on} \ \Gamma_{D}, \\ \boldsymbol{\nabla} \boldsymbol{u} \cdot \boldsymbol{n} + p \boldsymbol{n} &= \boldsymbol{g}_N \, \quad\;\; {\rm on} \ \Gamma_{N}. \\ \end{align*}

## Weak formulation

The weak formulation is derived by testing the first equation with a vector-valued function $\boldsymbol{v}$ in $H^1_0(\Omega)^d$ and the second equation with $q\in L^2(\Omega)$. Assuming, for the sake of simplicity, that $\boldsymbol{u}_D = \boldsymbol{0}$, and integrating by parts in the first equation, we end up with the saddle point problem
\begin{align*}
\int_{\Omega} \boldsymbol{\nabla} \boldsymbol{u} \cdot \boldsymbol{\nabla} \boldsymbol{v}
- (\boldsymbol{\nabla} \cdot \boldsymbol{v}) \ p \ {\rm d} x
&= \int_{\Omega} \boldsymbol{f} \cdot \boldsymbol{v} \, {\rm d} x + \int_{\Gamma_N} \boldsymbol{g} \cdot \boldsymbol{v} \, {\rm d} s. \\
\int_\Omega (\boldsymbol{\nabla} \cdot \boldsymbol{u}) \ q \ {\rm d} x &= 0
\end{align*}
The Stokes equations can easily be formulated in a "total" variational form; that is, a form where the two variables, the velocity and the pressure, are approximated simultaneously. Summing the two previous equations, we obtain the weak problem: Find $ (\boldsymbol{u}, p) \in W$ such that
\begin{equation*} a((\boldsymbol{u}, p), (\boldsymbol{v}, q)) = L((\boldsymbol{v}, q)) \end{equation*}
for all $(\boldsymbol{v}, q) \in W$, where
\begin{align*} a((\boldsymbol{u}, p), (\boldsymbol{v}, q)) &= \int_{\Omega} \boldsymbol{\nabla} \boldsymbol{u} \cdot \boldsymbol{\nabla} \boldsymbol{v} - (\boldsymbol{\nabla} \cdot \boldsymbol{v}) \ p + (\boldsymbol{\nabla} \cdot \boldsymbol{u}) \ q \, {\rm d} x, \\ L((\boldsymbol{v}, q)) &= \int_{\Omega} \boldsymbol{f} \cdot \boldsymbol{v} \, {\rm d} x + \int_{\Gamma_N} \boldsymbol{g} \cdot \boldsymbol{v} \, {\rm d} s. \\ \end{align*}
The space $W$ should be a product function space $W = H^1_0(\Omega)^d \times L^2(\Omega)$, such that $\boldsymbol{u} \in H^1_0(\Omega)^d$ and $q \in L^2(\Omega)$.

## Numerical solution with FEniCS

We want to solve the problem using the **stable pairs of finite elements spaces $\mathbb{P}^2/ \mathbb{P}^1$** on a uniform mesh sequence with an increasing number of subdivisions $n$. 
Then, we **verify numerically the convergence performances**.

In [ ]:
%%capture
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [ ]:
from fenics import *
import matplotlib.pyplot as plt
import numpy as np

After generating the mesh, we have to define the discrete spaces. In this case we built a "mixed finite element" (an **element pair**) consisting of **continuous piecewise quadratics (for the velocity) and continuous piecewise linears (for the pressure)** (called Taylor–Hood (TH) elements and is a stable pair for the Stokes equations).

In [ ]:
# Example for building a finite element pair in FEniCS
mesh = UnitSquareMesh(2, 2, 'crossed')
print(mesh.ufl_cell())

V = VectorElement('CG', mesh.ufl_cell(), 2)
Q = FiniteElement('CG', mesh.ufl_cell(), 1)
print(V,Q)

X = FunctionSpace(mesh, V*Q)
print(X)

triangle
<vector element with 2 components of <CG2 on a triangle>> <CG1 on a triangle>
FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), MixedElement(VectorElement(FiniteElement('Lagrange', triangle, 2), dim=2), FiniteElement('Lagrange', triangle, 1)))


In [ ]:
def solve_stokes(n, degree, u_exact, p_exact, f):
    # 1. mesh generation
    nx, ny = n, n
    mesh = UnitSquareMesh(nx, ny, 'crossed')

    # 2. finite element space
    V = VectorElement('CG', mesh.ufl_cell(), degree+1)
    Q = FiniteElement('CG', mesh.ufl_cell(), degree)
    #X = FunctionSpace(mesh, MixedElement([V, Q]))

    TH = V * Q
    X = FunctionSpace(mesh, TH)

    # 3. problem definition
    u, p = TrialFunctions(X)
    v, q = TestFunctions(X)

    def boundary(x, on_boundary):
          return on_boundary

    def origin(x):
        return near(x[0], 0.0) and near(x[1], 0.0)

    bc = DirichletBC(X.sub(0), u_exact, boundary)
    
    # to enforce the average value for the pressure with fix one boundary DOF 
    bc_pressure = DirichletBC(X.sub(1), p_exact(0, 0), origin, 'pointwise')

    a = (inner(grad(u), grad(v)) - p*div(v) - div(u)*q) * dx
    L = dot(f, v) * dx

    # 4. solution
    x = Function(X)
    solve(a == L, x, [bc, bc_pressure])

    u, p = x.split()

    u.rename('velocity', 'velocity')
    p.rename('pressure', 'pressure')

    return u, p

## Convergence analysis considering an analytical test case

Compute the functions $\boldsymbol{f}$ and $\boldsymbol{g}$ corresponding to the analytical solution
$$
            \boldsymbol{u}(x,y) = \begin{bmatrix}-\cos{x}\sin{y} \\ \sin{x}\cos{y}\end{bmatrix},
            \qquad
            p(x,y) = -\frac{1}{4}(\cos{(2x)}+\cos{(2y)}).
$$

Assess the convergence of the Taylor-Hood FE method, measuring the errors as 
$$
||\boldsymbol{u} - \boldsymbol{u}_h||_{H^1(\Omega)}, \quad\text{ and}\quad ||p - p_h||_ {L^2(\Omega)}.
$$

In [ ]:
u_exact = Expression((
        '-cos(x[0]) * sin(x[1])',
        'sin(x[0]) * cos(x[1])'
    ), degree=2)
p_exact = Expression(
    '-0.25 * (cos(2*x[0]) + cos(2*x[1]))',
    degree=2)
f = Expression((
        '-2 * cos(x[0]) * sin(x[1]) + 0.5 * sin(2 * x[0])',
        '2 * sin(x[0]) * cos(x[1]) + 0.5 * sin(2 * x[1])'
    ), degree=2)

for degree in [1, 2]:
  for n in [5, 10, 20, 40]:
    uh, ph = solve_stokes(n, degree, u_exact, p_exact, f)
    
    eL2 = errornorm(p_exact, ph, 'L2')
    eH1 = errornorm(u_exact, uh, 'H1')

    print('n={} degree={} eL2={:.2e} eH1={:.2e}'.format(n, degree, eL2, eH1))
  print()

n=5 degree=1 eL2=3.97e-03 eH1=1.60e-03
n=10 degree=1 eL2=1.01e-03 eH1=3.99e-04
n=20 degree=1 eL2=2.55e-04 eH1=9.97e-05
n=40 degree=1 eL2=6.38e-05 eH1=2.49e-05

n=5 degree=2 eL2=9.15e-05 eH1=3.26e-05
n=10 degree=2 eL2=7.12e-06 eH1=4.32e-06
n=20 degree=2 eL2=6.79e-07 eH1=5.54e-07
n=40 degree=2 eL2=7.55e-08 eH1=7.01e-08

